# BIDS Quality Check  
This script has options to do:    
* Volume check  
* Folder/file counts  

In [1]:
import glob, os
import pandas as pd
import pdb
import subprocess
from IPython.core import display as ICD
pd.set_option('display.max_colwidth', -1)
import numpy as np
pd.set_option('display.max_rows', 159)


In [2]:
bids_path = '/projects/niblab/bids_projects/Experiments/bbx/bids/'


In [3]:
scan_file ="bbx_scan_11.19.tsv"
report = pd.read_csv(scan_file,sep="\t")

report.columns = ['subject', 'BIDS check', 'fmriprep check', 'Anat', 'epi_r1',
   'sdc_r1', 'epi_r2', 'sdc_r2', 'epi_r3', 'sdc_r3',
   'epi_r4', 'sdc_r4', 'From TSV Files', 'fd_r1', 'min_r1',
   'max_r1', 'mcflirt_3mm_r1', 'fd_r2', 'min_r2',
   'max_r2', 'mcflirt_3mm_r2', 'fd_r3', 'min_r3',
   'max_r3', 'mcflirt_3mm_r3', 'fd_r4', 'min_r4',
   'max_r4', 'mcflirt_3mm_r4', 'FSL Level 1', 'activation_r1', 'activation_r2',
   'activation_r3', 'activation_r4', 'FINAL CHECK', 'Intials']


col_details = report.iloc[0:2, :]
col_details['fd_r1']
report.drop([0,1], inplace=True)
report = report.set_index(["subject"])



FileNotFoundError: [Errno 2] File b'bbx_scan_11.19.tsv' does not exist: b'bbx_scan_11.19.tsv'

In [ ]:

# get scan notes
df_w1_notes=pd.read_csv('w1_notes.csv', encoding='latin-1')
#df_w1_notes.head()
#df_w1_notes.columns
df_clean=df_w1_notes[['participantID', 'w1scan_scannotes']]

df_clean.set_index("participantID", inplace=True)
df_clean.index = df_clean.index.str.lower()
df_clean = df_clean.drop(['participant id (bbx_###)'])

## DICOM Check

In [ ]:

    
# Get DICOM id list
s1_dcms = [x.split("/")[-1].split("-")[1].lstrip("0") for x in
             glob.glob(os.path.join(bids_path, "sourcedata/DICOM/ses-1/sub-*"))]
s1_dcms = np.unique(np.array(s1_dcms)).tolist()
# get expected id list from notes
s1_sub_ids = [x.split("_")[1].lstrip('0') for x in df_clean.index.values]
s1_sub_ids = np.unique(np.array(s1_sub_ids)).tolist()

# get total count for DICOM and expected id lists
s1_sub_exp_ct = len(s1_sub_ids)
s1_sub_dcm_ct = len(s1_dcms)

# return the unique values in ar1 that are not in ar2

# ids found in dicom directory but not id list
s1_mia_id = np.setdiff1d(s1_dcms, s1_sub_ids)

# ids missing from dicom directories
s1_mia_dcm = np.setdiff1d(s1_sub_ids, s1_dcms)

s1_mia_id = s1_mia_id.tolist()
s1_mia_dcm = s1_mia_dcm.tolist()

# look at scan notes for any missing DICOM ids
for id_ in s1_mia_dcm:
    bbx_id = "bbx_{:03d}".format(int(id_))
    try:
        print("{}, notes: \n{}\n".format(bbx_id, df_clean.loc[bbx_id]))
    except:
        print("Missing scan notes for {} \n".format(bbx_id))
        


## Report Output
print("BBx Session 1 DICOM info:")
print("\nUnique DICOM directories found: {}".format(s1_sub_dcm_ct))
#print("\nScan notes subject count: {} \tUnique DICOM directories found: {}".format(s1_sub_exp_ct, s1_sub_dcm_ct))
#print("DICOM IDs available: \t",sorted(list(map(int, s1_sub_ids ))))



## BIDS Check  
Other:    
* BIDS validator  
* Volume Check
#### Compare DICOM ids with BIDS ids found, list any missing from BIDS

In [ ]:
s1_bids = np.unique(np.array([x.split("/")[-2].split("-")[1].lstrip("0") for x in
             glob.glob(os.path.join(bids_path, "sub-*/ses-1"))])).tolist()
s1_bids_ct = len(s1_bids)

# subjects found in DICOM list but not in bids
s1_mia_bids = np.setdiff1d(s1_dcms, s1_bids)

s1_bids = sorted(list(map(int,  s1_bids)))
## report
#print("BIDS Session 1 Info: ")
#print("Unique BIDS subject count: {} \n\n>>> Missing IDs: \t{} \n".format(s1_bids_ct, s1_mia_bids))
#print("BIDS IDs found: ", s1_bids)

#### Fill in a dictionary with subject ids and find their directories:  
#### *Value of 1 translates that the directory and files are missing*

In [4]:

# if no missing IDs found, that is good, meaning we have at least attempted to generate all of the subject directories available
# now we have to inspect individual directories
qa_dict={}
bids_dirs = glob.glob(os.path.join(bids_path, "sub-*/ses-1"))
for dir_path in sorted(bids_dirs):
    # get id
    sub_id = dir_path.split("/")[-2]
    
    # setup dict
    if sub_id not in qa_dict:
        qa_dict[sub_id] = {}
        
    # gather files
    func_runs=glob.glob(os.path.join(dir_path, "func/*run*nii.gz"))
    func_rest=glob.glob(os.path.join(dir_path, "func/*resting*nii.gz"))
    anat=glob.glob(os.path.join(dir_path, "anat/*.nii.gz"))
    fmaps=glob.glob(os.path.join(dir_path, "fmap/*.nii.gz"))
    #print("Analysis of {}:".format(sub_id))
    
    
    
    # Save the file count for each direcotry into the dictionary: 
    
    # Functional files:
    if not func_runs:
        qa_dict[sub_id]["func_train_ct"] = 0
    else:
        qa_dict[sub_id]["func_train_ct"] = len(func_runs)
        for func in func_runs:

            filename = func.split("/")[-1]

            fsl_cmd ="fslnvols {}".format(func)
            vol=subprocess.check_output(fsl_cmd, shell=True)
            vol=str(vol,'utf-8').strip()
            if "training" in func:
                run_id= filename.split("_")[3]
                expected_vol = 233
                col_name = run_id+"_vol"

            elif "rl" in func:
                expected_vol = 212
                col_name = run_id+"_vol"
            else:
                expected_vol = 147
                col_name = "rest_vol"

            qa_dict[sub_id][col_name] = vol   

            
    # Resting functional files:
    if not func_rest:
        qa_dict[sub_id]["func_rest_ct"] = 0
    else:
        qa_dict[sub_id]["func_rest_ct"] = len(func_rest)
        for func in func_rest:

            fsl_cmd ="fslnvols {}".format(func)
            vol=subprocess.check_output(fsl_cmd, shell=True)
            vol=str(vol,'utf-8').strip()

            qa_dict[sub_id]["resting"] = vol   

    # Fieldmap files:          
    if not fmaps:
        qa_dict[sub_id]["fmap_ct"] = 0
    else:
        qa_dict[sub_id]["fmap_ct"] = len(fmaps)
        

    # Anatomical files:            
    if not anat:
        qa_dict[sub_id]["anat_ct"] = 0
    else:
        qa_dict[sub_id]["anat_ct"] = len(anat)


    
print("\n> Completed dictionary build.")


> Completed dictionary build.


#### Build dataframe:  

In [5]:
bids_df = pd.DataFrame(qa_dict).T
ICD.display(bids_df)

,func_train_ct,run-1_vol,run-2_vol,run-3_vol,run-4_vol,func_rest_ct,resting,fmap_ct,anat_ct
sub-001,4,233,233,233,233,1,147,3,1
sub-002,4,233,233,233,233,1,147,3,1
sub-003,4,233,233,233,233,1,147,3,1
sub-004,4,233,233,233,233,1,147,3,1
sub-005,4,233,233,233,233,1,147,3,1
sub-006,4,233,233,233,233,1,147,3,1
sub-007,4,233,233,233,233,1,147,3,1
sub-008,4,233,233,233,233,1,147,3,1
sub-009,4,233,233,233,233,1,147,3,1
sub-010,4,233,233,233,233,1,147,3,1


In [8]:
bids_df_3 = bids_df[bids_df["func_train_ct"] == 3]


In [12]:
bids_runs3=bids_df_3.index.values
bids_runs3

array(['sub-048', 'sub-073', 'sub-076', 'sub-146', 'sub-147'],
      dtype=object)

In [13]:
bids_df[bids_df["func_train_ct"] < 3]


,func_train_ct,run-1_vol,run-2_vol,run-3_vol,run-4_vol,func_rest_ct,resting,fmap_ct,anat_ct
sub-060,2,233,233,NaN,NaN,0,NaN,3,1


#### Get subjects with missing files:

In [38]:
missing_df = bids_df[(bids_df == 0).any(axis="columns")]

In [39]:
ICD.display(missing_df)

,func_train_ct,run-1_vol,run-2_vol,run-3_vol,run-4_vol,func_rest_ct,resting,fmap_ct,anat_ct
sub-060,2,233,233,NaN,NaN,0,NaN,3,1
sub-076,3,233,233,233,NaN,0,NaN,3,1


#### Write dataframes to `.tsv` files:

In [40]:
bids_df.to_csv(os.path.join(bids_path, "derivatives/quality_check/bids_df_ct.tsv"), sep="\t")
missing_df.to_csv(os.path.join(bids_path, "derivatives/quality_check/bids_df_mia.tsv"), sep="\t")

#### Now lets try to get as much information about the missing subjects as we can:   



In [41]:
bids_ids_mia=list(missing_df.index.values)

In [42]:
print("\nMISSING SUBJECT IDS: \t {} \n".format(" ".join(bids_ids_mia)))
print("************************************************")

generate_report = False
if generate_report == True:
    for sub_id in missing_df.index.values:
        print("\nID: ",sub_id)
        dir_path = glob.glob(os.path.join(bids_path, sub_id, "ses-1/*/*"))
        print("\n> FILES FOUND: \n")
        for z in dir_path:
            print(z.split("/")[-1])
        try:
            print("\n> DIRECTORIES FOUND MISSING: ")
            ICD.display( missing_df.loc[sub_id])
            print("> SCAN NOTES (IF AVAILABLE):")
            ICD.display( df_clean.loc[sub_id.replace("sub-", "bbx_")])
        except:
            print("\nNo scan notes found")
        print("________________________________________________________________")


MISSING SUBJECT IDS: 	 sub-060 sub-076 

************************************************


In [43]:
# get stat variables for qc

bids_complete_ct=len(s1_bids)-len(bids_ids_mia)
expected_ids = set(s1_bids) - set([int(x.split('-')[1].lstrip('0')) for x in bids_ids_mia])
expected_ids=list(expected_ids)



## Notes:  
* sub-094 and sub-128 each had unqiue identifiers for heuristic file and have no functional runs with correct volumes, will drop subjects
* sub-155, *dropped before scans*  

## Fill in QC report

#### Load, format data

In [44]:
# Load original tsv report

tsv_report = "/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/quality_check/qc_report_s1.tsv"


# put into df
report_df_orig=pd.read_csv(tsv_report,sep="\t")
# rename columns 
report_df_orig.columns = ['subject', 'BIDS check', 'fmriprep check', 'Anat', 'epi_r1',
       'sdc_r1', 'epi_r2', 'sdc_r2', 'epi_r3', 'sdc_r3',
       'epi_r4', 'sdc_r4', 'From TSV Files', 'fd_r1', 'min_r1',
       'max_r1', 'mcflirt_3mm_r1', 'fd_r2', 'min_r2',
       'max_r2', 'mcflirt_3mm_r2', 'fd_r3', 'min_r3',
       'max_r3', 'mcflirt_3mm_r3', 'fd_r4', 'min_r4',
       'max_r4', 'mcflirt_3mm_r4', 'FSL Level 1', 'activation_r1', 'activation_r2',
       'activation_r3', 'activation_r4', 'FINAL CHECK', 'Intials']



In [45]:
# get column details
col_details = report_df_orig.iloc[0:2, :]
col_details['fd_r1']
#report_df_orig.drop([0,1], inplace=True)
report_df_orig = report_df_orig.set_index(["subject"])


In [46]:
template_report = report_df_orig.iloc[:159,:]
#report_df.head()

In [47]:
remove_ids = [index for index in template_report.index.values if int(index.split("-")[1]) not in expected_ids]
#for index in remove_ids

In [48]:
for index in remove_ids:
    try: 
        template_report.drop(index, inplace=True)
    except:
        pass
for x in expected_ids:
    index = "sub-{:03d}".format(x)
    funcs = glob.glob(os.path.join(bids_path, index, 'ses-1/func/*nii.gz'))
    if not funcs:
        print("MISSING FUNCS")
        pass
    else:
        print("Subject {}:\tPASSED anatomical check".format(x))
        template_report.loc[ index, 'BIDS check'] = "y"


Subject 1:	PASSED anatomical check
Subject 2:	PASSED anatomical check
Subject 3:	PASSED anatomical check
Subject 4:	PASSED anatomical check
Subject 5:	PASSED anatomical check
Subject 6:	PASSED anatomical check
Subject 7:	PASSED anatomical check
Subject 8:	PASSED anatomical check
Subject 9:	PASSED anatomical check
Subject 10:	PASSED anatomical check
Subject 11:	PASSED anatomical check
Subject 12:	PASSED anatomical check
Subject 13:	PASSED anatomical check
Subject 14:	PASSED anatomical check
Subject 15:	PASSED anatomical check
Subject 16:	PASSED anatomical check
Subject 17:	PASSED anatomical check
Subject 18:	PASSED anatomical check
Subject 19:	PASSED anatomical check
Subject 20:	PASSED anatomical check
Subject 21:	PASSED anatomical check
Subject 22:	PASSED anatomical check
Subject 23:	PASSED anatomical check
Subject 24:	PASSED anatomical check
Subject 25:	PASSED anatomical check
Subject 26:	PASSED anatomical check
Subject 27:	PASSED anatomical check
Subject 28:	PASSED anatomical check
S

In [49]:
template_report

,BIDS check,fmriprep check,Anat,epi_r1,sdc_r1,epi_r2,sdc_r2,epi_r3,sdc_r3,epi_r4,...,min_r4,max_r4,mcflirt_3mm_r4,FSL Level 1,activation_r1,activation_r2,activation_r3,activation_r4,FINAL CHECK,Intials
subject,,,,,,,,,,,,,,,,,,,,,
sub-001,y,--,good,good,good,good,good,good,good,good,...,0.036686,1.315115,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-002,y,--,good,good,good,good,good,good,good,good,...,0.041908,0.701858,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-003,y,--,good,good,good,good,good,good,good,good,...,0.035671,0.470508,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-004,y,--,good,good,good,good,good,good,good,good,...,0.024134,0.881343,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-005,y,--,good,good,good,good,good,good,good,good,...,0.021201,0.675988,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-006,y,--,good,good,good,good,good,good,good,good,...,0.029703,2.800106,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-007,y,--,good,good,good,good,good,good,good,good,...,0.036950,0.798493,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-008,y,--,good,good,good,good,good,good,good,good,...,0.016623,1.602760,pass,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-009,y,--,good,good,good,good,good,good,good,good,...,0.032103,0.400754,pass,--,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
template_report.to_csv("/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/quality_check/qc_report_s1.tsv", sep="\t")



In [51]:
## Report Output
print("BBx Session 1 DICOM info:")
print("\nUnique DICOM directories found: {}".format(s1_sub_dcm_ct))

# report
print("BIDS Session 1 Info: ")
print("Unique BIDS subject count: {} \n\n>>> Missing IDs: \t{} \n".format(s1_bids_ct, s1_mia_bids))
print("BIDS IDs found: ", s1_bids)

BBx Session 1 DICOM info:

Unique DICOM directories found: 141
BIDS Session 1 Info: 
Unique BIDS subject count: 141 

>>> Missing IDs: 	[] 

BIDS IDs found:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 104, 107, 108, 109, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 127, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 159]
